In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_21496\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_21496\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,Sugarcane,28-28
1,4,Millets,28-28
2,4,Millets,17-17-17
3,4,Barley,10-26-26
4,3,Paddy,DAP


In [11]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [12]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,28-28
1,4,4,28-28
2,4,4,17-17-17
3,4,0,10-26-26
4,3,6,DAP


In [13]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [14]:
df_train1 = pd.concat([df.num,df.cat],axis=1)

In [15]:
df_train1.head()

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type,Crop Type,Fertilizer Name
0,0,37,70,36,36,4,5,1,8,4
1,1,27,69,65,30,6,18,4,4,4
2,2,29,63,32,24,12,16,4,4,2
3,3,35,62,54,39,12,4,4,0,0
4,4,35,58,43,37,2,16,3,6,5


In [16]:
import tensorflow as tf

In [17]:
from tensorflow import keras

In [18]:
X = df_train1[["Humidity","Moisture","Soil Type","Crop Type"]]

In [19]:
y = df_train1[["Fertilizer Name"]]

In [20]:
from tensorflow.keras.regularizers import l2

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
num_classes = len(np.unique(y_train)) 

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeUniform

model = Sequential([
    Dense(128, activation='relu', kernel_initializer=HeUniform()),
    Dense(64, activation='relu'),
    Dense(7, activation='softmax')
])
optimizer = Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

logits = model.predict(X[:10])
print("Logits:\n", logits)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Logits:
 [[2.0830660e-35 1.1718866e-20 0.0000000e+00 7.6851677e-03 0.0000000e+00
  8.7078828e-23 9.9231488e-01]
 [0.0000000e+00 1.4099268e-35 0.0000000e+00 1.0000000e+00 0.0000000e+00
  0.0000000e+00 7.9784360e-11]
 [1.8687780e-29 1.7480270e-17 0.0000000e+00 2.2339849e-04 0.0000000e+00
  3.8250483e-20 9.9977666e-01]
 [0.0000000e+00 5.6215545e-27 0.0000000e+00 9.9999833e-01 0.0000000e+00
  1.4495821e-30 1.6606180e-06]
 [1.2231204e-35 3.0873916e-22 0.0000000e+00 9.9548137e-01 0.0000000e+00
  3.0643075e-25 4.5186551e-03]
 [2.3457429e-28 9.3037642e-17 0.0000000e+00 2.0264385e-02 0.0000000e+00
  3.9588616e-19 9.7973561e-01]
 [0.0000000e+00 3.0514127e-29 0.0000000e+00 9.9999964e-01 0.0000000e+00
  3.8762370e-32 3.6165844e-07]
 [5.2814482e-37 1.4461605e-22 0.0000000e+00 9.7921526e-01 0.0000000e+00
  7.0651061e-26 2.0784764e-02]
 [3.6243557e-30 1.0760379e-17 0.0000000e+00 9.5627886e-01 0.0000000e+00
  1.2672569e-20 4.3721203e-02]
 [9.1136553e-32 2.9362914

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)


In [26]:
# If y_train is a DataFrame
y_array = y_train['Fertilizer Name'].values  # replace with your actual column name

# Then compute class weights
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_array)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_array)

In [27]:
from tensorflow.keras.utils import to_categorical

y_encoded = to_categorical(y, num_classes=7)
model.fit(X, y_encoded, epochs=3)

Epoch 1/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.1464 - loss: 2.3576
Epoch 2/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.1530 - loss: 1.9433
Epoch 3/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.1525 - loss: 1.9433


In [28]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=3)
model.evaluate(X, y)  # works directly with integer labels

Epoch 1/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step - accuracy: 0.1511 - loss: 1.9425
Epoch 2/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - accuracy: 0.1511 - loss: 1.9425
Epoch 3/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step - accuracy: 0.1522 - loss: 1.9426
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 23s 957us/step - accuracy: 0.1528 - loss: 1.9427


[1.9426554441452026, 0.1525813341140747]

In [29]:
# Sanity check on label spread
print(np.unique(y, return_counts=True))

# Try predicting from untrained model
sample_preds = model.predict(X[:112])
print("Predictions:", sample_preds.argmax(axis=1))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step 
Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]


In [30]:
import numpy as np
import random
from collections import defaultdict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [31]:
# --- Step 1: Reservoir Sampling ---
from collections import defaultdict
import random

def class_based_reservoir_sampling(stream, k):
    """
    Maintain separate reservoirs per class label.
    :param stream: iterable of (sample, label) tuples
    :param k: buffer size per class
    :return: dict of class-wise reservoirs
    """
    reservoirs = defaultdict(list)
    class_counts = defaultdict(int)  # counts per class for reservoir logic

    for sample, label in stream:
        count = class_counts[label]
        buffer = reservoirs[label]

        if count < k:
            buffer.append(sample)
        else:
            j = random.randint(0, count)
            if j < k:
                buffer[j] = sample

        class_counts[label] += 1

    return reservoirs

In [32]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

C:\Users\pkitt\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
# --- Step 2: Build Balanced Dataset ---
# Assuming fertilizer dataset as (features, fertilizer_label)
reservoirs = class_based_reservoir_sampling(zip(X, y_encoded), k=50)


In [34]:
# Check what labels got into the buffer
print("Unique labels in y_balanced:", set(y_encoded))

Unique labels in y_balanced: {0, 1, 2, 3, 4, 5, 6}


In [35]:
# --- Step 3: Model Definition & Training ---

model = Sequential([
    Dense(128, activation='relu',),
    Dense(64, activation='relu'),
    Dense(7, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)

In [36]:
model.fit(X, y_encoded, epochs=3)
model.evaluate(X, y_encoded)

Epoch 1/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step - accuracy: 0.1498 - loss: 1.9737
Epoch 2/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.1521 - loss: 1.9426
Epoch 3/3
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.1513 - loss: 1.9427
23438/23438 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.1517 - loss: 1.9427


[1.9427213668823242, 0.15184932947158813]

In [37]:
# --- Step 4: Predict & Report ---
from collections import Counter
print("Class distribution:", Counter(y_encoded))

Class distribution: Counter({1: 114436, 0: 113887, 2: 112453, 4: 111158, 3: 110889, 5: 94860, 6: 92317})
